In [1]:
%store -r df_German_train

In [2]:
df_German_train.head()

,#,Answers,Class,Subclass,answer_len,word_count,language
181494,15471114907050152515K20,Sie versucht der anderen Person zu Helfen Löse...,M,2,146,23,de
69605,3241342197844644BG,"sie möchte immer höher hinaus, sie gibt alles ...",L,2,110,18,de
129925,6581431170681727K,"Sie will verhindern, dass Fehler erneut auftre...",M,4,99,15,de
104570,10691401290219549K,"Ich habe das Gefühl, diese Person bräuchte ein...",A,5,146,22,de
68925,1361353255679140M9,"Problem lösen, erklären was passiert ist. schl...",M,5,101,15,de


In [3]:
import os, codecs
from time import time
import sys
t0 = time()

# to use my own preprocessing in the vectorizer later
def my_dummy(doc):
    return doc

# Preprocessing

In [4]:
from nltk.corpus import stopwords
#download stopwords, add 'dass' for new spellins 
stopWords = set(stopwords.words('german'))
stopWords.add('dass')

In [ ]:
import re
import numpy as np
import nltk
from nltk import word_tokenize





#tokenizes a list of strings
#return: list of tokens
def tokenize_text(text):
    tokens = nltk.word_tokenize(text, language = 'german')
    return tokens

#retrns: list of strings which do not occure in stopWords
def remove_stopwords(text):
    return [w for w in text if not w in stopWords]


pattern = re.compile("\-|\!|\:|\.|\d|\,|\:|\(|\)|\?|\"|\\|\+|\%|\/")
def remove_patterns_from_string(strg):
    return re.sub(pattern, " ", strg)

# in : row for row from vector
# out: preprocessed version of the row
def preprocess_text(corpus):
        helplist = []
        for doc in corpus:
            #only user lower case
            doc = doc.lower()
            doc = remove_patterns_from_string(doc)
            doc = tokenize_text(doc)
            doc = remove_stopwords(doc)
            helplist.append(doc)
        return helplist

#split it into X ( = input) and y ( the labels)
X_train, y_train = preprocess_text(df_German_train["Answers"]), df_German_train["Class"]

import random
print(random.sample(X_train, 100))

In [8]:
teste = ['ich bin ein testcorpus']
print(preprocess_text(teste))

[['testcorpus']]


In [18]:
%store -r df_German_test
X_test, y_test = preprocess_text(df_German_test["Answers"]), df_German_test["Class"]

In [43]:
%whos

Variable                      Type                          Data/Info
---------------------------------------------------------------------
CountVectorizer               type                          <class 'sklearn.feature_e<...>on.text.CountVectorizer'>
FeatureUnion                  ABCMeta                       <class 'sklearn.pipeline.FeatureUnion'>
GridSearchCV                  ABCMeta                       <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
LinearSVC                     type                          <class 'sklearn.svm.classes.LinearSVC'>
MultinomialNB                 ABCMeta                       <class 'sklearn.naive_bayes.MultinomialNB'>
Pipeline                      ABCMeta                       <class 'sklearn.pipeline.Pipeline'>
TfidfVectorizer               type                          <class 'sklearn.feature_e<...>on.text.TfidfVectorizer'>
X_test                        list                          n=62820
X_train                       list          

# Start of the ML Part.

In [20]:
from sklearn.metrics import classification_report
import pandas as pd

# minimal classification performance: all answers set to M ( ZeroN - Classifier)
#
m_labels = []
for i in range(len(y_test)):
    m_labels.append("M")
powermotive = pd.Series(m_labels)
print(classification_report(y_test, powermotive))

C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2970
           A       0.00      0.00      0.00     10607
           L       0.00      0.00      0.00     12270
           M       0.59      1.00      0.74     36973

   micro avg       0.59      0.59      0.59     62820
   macro avg       0.15      0.25      0.19     62820
weighted avg       0.35      0.59      0.44     62820



In [21]:
# This cell trains SVC classifier with different features (tf, tfidf, min/max _docfrequency, kernels, C, iterations )
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline, FeatureUnion

pipeline1 = []
grid_search1 = []
print("start classification with SVC...")
#define pipeline combining feature extractor with classifier
pipeline1 = Pipeline([
    ('vect1', CountVectorizer(preprocessor = my_dummy,
                              tokenizer =my_dummy, 
                              token_pattern=None)),                     
    ('clf1', LinearSVC()),    
])

parameters1 = {
    'vect1__max_df': (1.0,0.9,0.8, 0.5),
    'vect1__ngram_range':((1,1),(1,2)),
    'vect1__min_df': (1,5,10),
    'vect1__max_features': (10000,20000,30000, None), 
    'clf1__C':(0.1,),
    'clf1__max_iter':(300,),
}

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search1 = GridSearchCV(pipeline1, parameters1, cv=5,
                           n_jobs = -1, verbose = 1, scoring='accuracy')
    print("performing gridsearch...")
    time_svc = time()
    grid_search1.fit(X_train, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search1.best_score_)
    print("best parameters set:")
    best_parameters = grid_search1.best_estimator_.get_params()
    for param_name in sorted(parameters1.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    predictions = grid_search1.predict(X_test)
    print(classification_report(y_test, predictions))


start classification with SVC...
performing gridsearch...
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 21.9min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 51.3min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 56.4min finished


done in 3399.514s

Best score: 0.827
best parameters set:
	clf1__C: 0.1
	clf1__max_iter: 300
	vect1__max_df: 1.0
	vect1__max_features: None
	vect1__min_df: 1
	vect1__ngram_range: (1, 2)
              precision    recall  f1-score   support

           0       0.56      0.22      0.31      2970
           A       0.80      0.77      0.78     10607
           L       0.81      0.77      0.79     12270
           M       0.85      0.91      0.88     36973

   micro avg       0.83      0.83      0.83     62820
   macro avg       0.75      0.67      0.69     62820
weighted avg       0.82      0.83      0.82     62820



In [24]:
# This cell trains SVC classifier with different features (tf, tfidf, min/max _docfrequency, kernels, C, iterations )
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline, FeatureUnion

pipeline1 = []
grid_search1 = []
print("start classification with SVC...")
#define pipeline combining feature extractor with classifier
pipeline1 = Pipeline([
    ('vect1', TfidfVectorizer(preprocessor = my_dummy,
                              tokenizer =my_dummy, 
                              token_pattern=None)),
    ('clf1', LinearSVC()),    
])

parameters1 = {
    'vect1__max_df': (0.9,),
    'vect1__ngram_range':((1,1),),
    'vect1__min_df': (5,10,15),
    'vect1__max_features': (None,), 
    'vect1__use_idf': (False, True),
    'clf1__C':(0.1,),
    'clf1__max_iter':(300,),
}

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search1 = GridSearchCV(pipeline1, parameters1, cv=5,
                           n_jobs = -1, scoring='accuracy')
    print("performing gridsearch...")
    time_svc = time()
    grid_search1.fit(X_train, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search1.best_score_)
    print("best parameters set:")
    best_parameters = grid_search1.best_estimator_.get_params()
    for param_name in sorted(parameters1.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    predictions = grid_search1.predict(X_test)
    print(classification_report(y_test, predictions))


start classification with SVC...
performing gridsearch...
done in 171.799s

Best score: 0.827
best parameters set:
	clf1__C: 0.1
	clf1__max_iter: 300
	vect1__max_df: 0.9
	vect1__max_features: None
	vect1__min_df: 5
	vect1__ngram_range: (1, 1)
	vect1__use_idf: True
              precision    recall  f1-score   support

           0       0.65      0.15      0.24      2970
           A       0.82      0.75      0.78     10607
           L       0.82      0.75      0.79     12270
           M       0.83      0.93      0.88     36973

   micro avg       0.83      0.83      0.83     62820
   macro avg       0.78      0.64      0.67     62820
weighted avg       0.82      0.83      0.81     62820



In [28]:
# This cell trains SVC classifier with different features (tf, tfidf, min/max _docfrequency, kernels, C, iterations )
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.pipeline import Pipeline

pipeline1 = []
grid_search1 = []
print("start classification with SVC...")
#define pipeline combining feature extractor with classifier
pipeline1 = Pipeline([
    ('vect1', TfidfVectorizer(preprocessor = my_dummy,
                              tokenizer =my_dummy, 
                              token_pattern=None)),
    ('clf1', xgb.XGBClassifier()),    
])

parameters1 = {
    'vect1__max_df': (1.0, 0.9),
    'vect1__ngram_range':((1,1),),
    'vect1__min_df': (5,),
    'vect1__max_features': (None, 20000,), 
    'vect1__use_idf': (True, False),
    'clf1__objective':['multi:softmax',],
    'clf1__num_class':[4],
    'clf1__learning_rate': [0.1], #so called `eta` value
    'clf1__max_depth': [6],
    'clf1__min_child_weight': [11],
    'clf1__silent': [1],
    'clf1__subsample': [0.8],
    'clf1__colsample_bytree': [0.7],
    'clf1__n_estimators': [1000],
}

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search1 = GridSearchCV(pipeline1, parameters1, cv=5,
                           n_jobs = -1, verbose = 1, scoring='accuracy')
    print("performing gridsearch...")
    time_svc = time()
    grid_search1.fit(X_train, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search1.best_score_)
    print("best parameters set:")
    best_parameters = grid_search1.best_estimator_.get_params()
    for param_name in sorted(parameters1.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    predictions = grid_search1.predict(X_test)
    print(classification_report(y_test, predictions))


start classification with SVC...
performing gridsearch...
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 192.7min finished


done in 12317.911s

Best score: 0.818
best parameters set:
	clf1__colsample_bytree: 0.7
	clf1__learning_rate: 0.1
	clf1__max_depth: 6
	clf1__min_child_weight: 11
	clf1__n_estimators: 1000
	clf1__num_class: 4
	clf1__objective: 'multi:softprob'
	clf1__silent: 1
	clf1__subsample: 0.8
	vect1__max_df: 1.0
	vect1__max_features: None
	vect1__min_df: 5
	vect1__ngram_range: (1, 1)
	vect1__use_idf: False
              precision    recall  f1-score   support

           0       0.58      0.19      0.29      2970
           A       0.83      0.72      0.77     10607
           L       0.82      0.73      0.77     12270
           M       0.82      0.93      0.87     36973

   micro avg       0.82      0.82      0.82     62820
   macro avg       0.76      0.64      0.68     62820
weighted avg       0.81      0.82      0.81     62820



mnb classifier

In [29]:
# This cell trains SVC classifier with different features (tf, tfidf, min/max _docfrequency, kernels, C, iterations )
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion

pipeline1 = []
grid_search1 = []
print("start classification with SVC...")
#define pipeline combining feature extractor with classifier
pipeline1 = Pipeline([
    ('vect1', CountVectorizer(preprocessor = my_dummy,
                              tokenizer =my_dummy, 
                              token_pattern=None)),                     
    ('clf1', MultinomialNB()),    
])

parameters1 = {
    'vect1__max_df': (1.0,0.9,0.8, 0.5),
    'vect1__ngram_range':((1,1),(1,2)),
    'vect1__min_df': (1,5,10),
    'vect1__max_features': (10000,20000,30000, None), 
    'clf1__alpha': [0.001, 0.01, 0.1, 0.5, 1,]
}

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search1 = GridSearchCV(pipeline1, parameters1, cv=5,
                           n_jobs = -1, verbose = 1, scoring='accuracy')
    print("performing gridsearch...")
    time_svc = time()
    grid_search1.fit(X_train, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search1.best_score_)
    print("best parameters set:")
    best_parameters = grid_search1.best_estimator_.get_params()
    for param_name in sorted(parameters1.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    predictions = grid_search1.predict(X_test)
    print(classification_report(y_test, predictions))


start classification with SVC...
performing gridsearch...
Fitting 5 folds for each of 480 candidates, totalling 2400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 46.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 83.0min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 132.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 188.6min
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed: 252.5min finished


done in 15155.893s

Best score: 0.820
best parameters set:
	clf1__alpha: 0.5
	vect1__max_df: 1.0
	vect1__max_features: None
	vect1__min_df: 1
	vect1__ngram_range: (1, 2)
              precision    recall  f1-score   support

           0       0.77      0.09      0.17      2970
           A       0.78      0.77      0.78     10607
           L       0.79      0.77      0.78     12270
           M       0.84      0.91      0.87     36973

   micro avg       0.82      0.82      0.82     62820
   macro avg       0.79      0.64      0.65     62820
weighted avg       0.82      0.82      0.81     62820



# short helper for colab

In [30]:
from sklearn.model_selection import train_test_split
X_biterms1, X_biterms2, y_biterms1, y_biterms2 = train_test_split(X_train, y_train, test_size = 0.5)
len(X_biterms1)

73289

In [31]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from biterm.utility import vec_to_biterms
def my_dummy(doc):
    return doc

if __name__ == "__main__":

    # vectorize texts
    vec = CountVectorizer(min_df = 4, max_df = 0.8, preprocessor = my_dummy,
                                                              tokenizer =my_dummy, 
                                                              token_pattern=None)
    Xbtm = vec.fit_transform(X_biterms1).toarray()

    # get vocabulary
    vocab = np.array(vec.get_feature_names())

    # get biterms
    biterms = vec_to_biterms(Xbtm)


In [39]:
del vocab, Xbtm, X_biterms1, X_biterms2

NameError: name 'vocab' is not defined

In [40]:
%whos

Variable                      Type                          Data/Info
---------------------------------------------------------------------
CountVectorizer               type                          <class 'sklearn.feature_e<...>on.text.CountVectorizer'>
FeatureUnion                  ABCMeta                       <class 'sklearn.pipeline.FeatureUnion'>
GridSearchCV                  ABCMeta                       <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
LinearSVC                     type                          <class 'sklearn.svm.classes.LinearSVC'>
MultinomialNB                 ABCMeta                       <class 'sklearn.naive_bayes.MultinomialNB'>
Pipeline                      ABCMeta                       <class 'sklearn.pipeline.Pipeline'>
TfidfVectorizer               type                          <class 'sklearn.feature_e<...>on.text.TfidfVectorizer'>
X_test                        list                          n=62820
X_train                       list          